In [1]:
import plotly.express as px
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import pandas as pd
import numpy as np
import glob
import json

In [2]:
%load_ext autoreload
%autoreload 2

import RDlite.accessor as ac

In [3]:
from RDlite import all_features

In [4]:
all_features

['company',
 'collection',
 'genre',
 'keyword',
 'rating',
 'language',
 'country',
 'name',
 'character',
 'gender',
 'movie',
 'adult',
 'budget',
 'original_language',
 'original_title',
 'overview',
 'popularity',
 'release_date',
 'revenue',
 'runtime',
 'status',
 'video',
 'vote_average',
 'vote_count']

In [5]:
ac.agg(*all_features, peek=True)

,language,adult,budget,imdb_id,original_language,original_title,overview,popularity,poster_path,release_date,...,title,video,vote_average,vote_count,keyword,company,rating,collection,genre,country
id,,,,,,,,,,,,,,,,,,,,,
862,English,False,30000000.0,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,1995-10-30,...,Toy Story,False,7.7,5415.0,jealousy,Pixar Animation Studios,NaN,Toy Story Collection,Animation,United States of America
8844,English,False,65000000.0,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,1995-12-15,...,Jumanji,False,6.9,2413.0,board game,TriStar Pictures,NaN,NaN,Adventure,United States of America
15602,English,False,0.0,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.712900,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,1995-12-22,...,Grumpier Old Men,False,6.5,92.0,fishing,Warner Bros.,NaN,Grumpy Old Men Collection,Romance,United States of America
31357,English,False,16000000.0,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,1995-12-22,...,Waiting to Exhale,False,6.1,34.0,based on novel,Twentieth Century Fox Film Corporation,NaN,NaN,Comedy,United States of America
11862,English,False,0.0,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,/e64sOI48hQXyru7naBFyssKFxVd.jpg,1995-02-10,...,Father of the Bride Part II,False,5.7,173.0,baby,Sandollar Productions,NaN,Father of the Bride Collection,Comedy,United States of America
949,English,False,60000000.0,tt0113277,en,Heat,"Obsessive master thief, Neil McCauley leads a ...",17.924927,/zMyfPUelumio3tiDKPffaUpsQTD.jpg,1995-12-15,...,Heat,False,7.7,1886.0,robbery,Regency Enterprises,NaN,NaN,Action,United States of America
11860,Français,False,58000000.0,tt0114319,en,Sabrina,An ugly duckling having undergone a remarkable...,6.677277,/jQh15y5YB7bWz1NtffNZmRw0s9D.jpg,1995-12-15,...,Sabrina,False,6.2,141.0,paris,Paramount Pictures,NaN,NaN,Comedy,Germany
45325,English,False,0.0,tt0112302,en,Tom and Huck,"A mischievous young boy, Tom Sawyer, witnesses...",2.561161,/sGO5Qa55p7wTu7FJcX4H4xIVKvS.jpg,1995-12-22,...,Tom and Huck,False,5.4,45.0,NaN,Walt Disney Pictures,NaN,NaN,Action,United States of America
9091,English,False,35000000.0,tt0114576,en,Sudden Death,International action superstar Jean Claude Van...,5.231580,/eoWvKD60lT95Ss1MYNgVExpo5iU.jpg,1995-12-22,...,Sudden Death,False,5.5,174.0,terrorist,Universal Pictures,NaN,NaN,Action,United States of America
